In [30]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import glob
import os
from math import log

In [31]:
## Data Taken at 1000V on the PMT

allFiles = sorted(glob.glob("*.txt"),key=os.path.getmtime) # includes all files into a single stucture,
# Sorted by modification time so that the first measurement is first...
print(allFiles)

['RealSHGData_LaserDepend_Background_07092017.txt', 'RealSHGData_LaserDepend_98mW_07092017.txt', 'RealSHGData_LaserDepend_75mW_07092017.txt', 'RealSHGData_LaserDepend_70mW_07092017.txt', 'RealSHGData_LaserDepend_60mW_07092017.txt', 'RealSHGData_LaserDepend_80mW_07092017.txt', 'RealSHGData_LaserDepend_85mW_07092017.txt', 'RealSHGData_LaserDepend_90mW_07092017.txt']


In [32]:
frame = pd.DataFrame()
df = pd.DataFrame()

In [33]:
for i,file_ in enumerate(allFiles):
#    f = open(file_,'r')
#     for j,line in enumerate(f):
#         if j == 1:
#             date = line[14:22]
#             break
    df = pd.read_csv(file_,index_col = None, header=None, skiprows = 10, skipfooter = 1, delimiter = '\t')
#     frame[file_] = df[5]
    df['file']=file_
    df['timeDelay']= df[1]
    df['PDvalue'] = df[3]
#     df['date']=date
    if i < 1:
        frame = df
    else:
        frame = frame.append(df)

/Users/sanjaykhatri/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skip_footer; you can avoid this warning by specifying engine='python'.


In [34]:
c = 3*10**11; #Speed of Light in mm/s, Going to use this to convert spatial information to time. 
T1 = (718*0.000125 + 0.0229)*2*25.4/(10**-12*c)
def ConvertToTimeDelay(x,itr,offset):
    c = 3*10**11
    timeDelay = []
    for i in range(0,len(x)):
        timedelay = -((x[i]*itr + offset)*25.4*2/(10**-12*c))+T1
        timeDelay.append(timedelay)
    return(timeDelay)

In [35]:
times = pd.Series()
PDvalues = pd.Series()
AvgPD50 = pd.Series()
itr = 0.000125
offset = 0.023
for i in range(0,200):
    time = frame[1][i]
    PDvalue = frame[3][i]
    times = times.append(time)
    PDvalues = PDvalues.append(PDvalue)
#print(len(PDvalues))
    
times = times.values.reshape(200,len(allFiles))
PDvalues = PDvalues.values.reshape(200,len(allFiles)) 
timeDelay = ConvertToTimeDelay(times,itr,offset) # function converts to timedelay
MeanPD = PDvalues.mean(axis = 0) #Getting mean of all the data
StDevPD = PDvalues.std(axis = 0) # Getting Standard Deviation of the data
ErrOfMean = (StDevPD)/(200**0.5) # Standard error of mean stdev/sqrt(#ofdatapoints)
Background = MeanPD[0] # Background is the last element in the allFiles
ActualPD = np.absolute(MeanPD - Background)
RealPD = np.delete(ActualPD,0)
LogPD = np.log10(RealPD)
LaserPower = np.array([98,75,70,60,80,85,90])
LogLaser = np.log10(LaserPower)
error = np.delete(ErrOfMean,2)

In [36]:
Delta = len(LogPD)*sum(LogLaser**2) - (sum(LogLaser))**2
A = (sum(LogLaser**2)*sum(LogPD) - sum(LogLaser)*sum(LogLaser*LogPD))/Delta
B = (len(LogPD)*sum(LogLaser*LogPD) - (sum(LogPD)*sum(LogLaser)))/Delta
# Least Squares Fitting Parameters above
print(A)
print(B)
# Errors in Least Squares Plotting
err = (((len(LogPD)-2)**-1) * sum((LogPD - A -B*LogLaser)**2)**0.5)
errorA = err*(sum(LogLaser**2)/Delta)**0.5
errorB = err*(len(LogPD)/Delta)**0.5
print(errorA)
print(errorB)


-5.08843452903
1.94512665454
0.168314400626
0.0886909722814


In [37]:
fit = np.polyfit(LogLaser,LogPD,1,w=error/(log(10)*RealPD))
f = np.poly1d(fit)
pPower = np.linspace(LogLaser[1],LogLaser[-1],100)
f_new = f(pPower)
print(f_new)
print(f)

[-1.44362619 -1.44192482 -1.44022344 -1.43852206 -1.43682069 -1.43511931
 -1.43341793 -1.43171656 -1.43001518 -1.42831381 -1.42661243 -1.42491105
 -1.42320968 -1.4215083  -1.41980693 -1.41810555 -1.41640417 -1.4147028
 -1.41300142 -1.41130004 -1.40959867 -1.40789729 -1.40619592 -1.40449454
 -1.40279316 -1.40109179 -1.39939041 -1.39768904 -1.39598766 -1.39428628
 -1.39258491 -1.39088353 -1.38918215 -1.38748078 -1.3857794  -1.38407803
 -1.38237665 -1.38067527 -1.3789739  -1.37727252 -1.37557115 -1.37386977
 -1.37216839 -1.37046702 -1.36876564 -1.36706426 -1.36536289 -1.36366151
 -1.36196014 -1.36025876 -1.35855738 -1.35685601 -1.35515463 -1.35345325
 -1.35175188 -1.3500505  -1.34834913 -1.34664775 -1.34494637 -1.343245
 -1.34154362 -1.33984225 -1.33814087 -1.33643949 -1.33473812 -1.33303674
 -1.33133536 -1.32963399 -1.32793261 -1.32623124 -1.32452986 -1.32282848
 -1.32112711 -1.31942573 -1.31772436 -1.31602298 -1.3143216  -1.31262023
 -1.31091885 -1.30921747 -1.3075161  -1.30581472 -1.30

In [44]:
plt.plot(LogLaser, B*LogLaser + A, 'k',label = 'Fit: y = 1.945x-5.09')
plt.plot(LogLaser,LogPD,'b*',label = 'Data')
plt.plot()
plt.grid(True)
plt.xlabel('Log(Laser Power)')
plt.ylabel('Log(PMT Signal)')
plt.legend(loc = 'upper left')
#plt.show()
plt.savefig('LaserVsPMTSignal_07_09_2017.png')